<a href="https://colab.research.google.com/github/ieg-dhr/DigiKAR/blob/main/JupyterNotebooks_DigiKAR/Factoids_Step2a_VerticalConsolidation_Profs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a script for consolidating factoid lists in AP3.

The package mainly uses the Pandas package in Python to read and manipulate EXCEL data as DataFrames. DataFrames are 2-dimensional data representations in rows and columns. They can be written to different file formats such as CSV, EXCEL, JSON or RDF.

First of all, we need to connect this Colab notebook with your Google Drive and define the directory for input and output data.


In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

In the second step, we have to install additional Packages needed for working with CSV, EXCEL and DataFrames.

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install xlsxwriter
!pip install pandas
!pip install numpy

In **step 1**, we can import the packages to the script and load our data. Before merging the input files, names will be normalised as some have access spaces, capitalised surnames, or inverted first and last names.

The combined data will be written to a new dataframe and displayed.

In [ ]:
import xlsxwriter
import csv
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import re

# path to input files

factoid_path_API="https://github.com/ieg-dhr/DigiKAR/blob/main/Sample%20Data/Profs_API_factoid_consolidated_v1.xlsx?raw=true"
factoid_path_OCR="https://github.com/ieg-dhr/DigiKAR/blob/main/Sample%20Data/Profs_OCR_factoid_with-dates-and-institutions.xlsx?raw=true"

factoid_paths=["https://github.com/ieg-dhr/DigiKAR/blob/main/Sample%20Data/Profs_API_factoid_consolidated_v1.xlsx?raw=true",
               "https://github.com/ieg-dhr/DigiKAR/blob/main/Sample%20Data/Profs_OCR_factoid_with-dates-and-institutions.xlsx?raw=true"]

# define dataframe for final output

f_to_add=[]

# structure of input files

# obligatory columns in valid factoid list

column_names = ["factoid_ID",
                "pers_ID",
                "alternative_names",
                "event_type",
                "event_after-date",
                "event_before-date",
                "event_start",
                "event_end",
                "event_date",
                "pers_title",
                "pers_function",
                "place_name",
                "inst_name",
                "rel_pers",
                "source_quotations",
                "additional_info",
                "comment",
                "info_dump",
                "source",
                "source_site"]

frame_list=[]
for file in factoid_paths:
    df = pd.read_excel(file, index_col=None, dtype=str) # axis=1, sort=False sheet_name='FactoidList'
    df = df.fillna("n/a") # replace empty fields for string
    df_length=len(df)
    person_cleaned=[]

    try:
      for x in range(0, df_length):
          print(df_length - x)
          e_df=df.iloc[[x]].fillna("@") # virtual value to avoid issues with empty data frames

          # CLEAN PERSON NAMES

          pers_name=e_df['pers_name'].values[0]
          print(pers_name)
          if "," in pers_name:
            pers_new=pers_name.split(",")
            first_name=pers_new[1].strip()
            #print(first_name)
            last_name=pers_new[0].title().strip() # change "all caps" to sentence case
            #print(last_name)
            name_list=(first_name, last_name)

            try:
              name_reversed=" ".join(name_list) # religious titles are being ignored
              #print("This is the new name: ", name_reversed)
            except Exception as e:
              print(e)
          else:
            name_reversed=re.sub("\s\s+" , " ", pers_name)
          #print(name_reversed)
          person_cleaned.append(name_reversed)

    except IndexError:
      print("No more names found.")

    # add column with cleaned person names

    print(person_cleaned)
    print(len(person_cleaned))
    df['pers_cleaned']=person_cleaned

    frame_list.append(df)

f = pd.concat(frame_list, axis=0, ignore_index=True, sort=False)

print("There are ", len(f), "items in your DataFrame!")

# delete all duplicate rows with exact matches

f_unique=f.drop_duplicates()
print("Your DataFrame has now ", len(f_unique), "items with at least one unique cell." )

display(f_unique)

workbook=directory+'FACTOIDS_consolidated/Factoid_PROFS_merged_names-fixed.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
f_unique.to_excel(writer, sheet_name='ProfFactALL') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")



In **step 2**, we create dictionaries that define event successions. Some events start and end other events. Where the events started or ending or not yet made explicit, they are added to the dataframe in **step 3**.




In [ ]:
# create dictionaries with coded event relationships

event_starting_dict= {"Immatrikulation":"Studium",
                  "Graduation":"Studium",
                  "Aufnahme":"Mitgliedschaft",
                  "Rezeption":"Mitgliedschaft",
                  "Zulassung":"Studium",
                  "Aufschwörung":"Funktionssausübung",
                  "Introduktion":"Mitgliedschaft",
                  "Vokation":"Funktionsausübung",
                  "Ernennung":"Funktionsausübung",
                  "Amtseinführung":"Funktionsausübung",
                  "Ernennung":"Funktionsausübung",
                  "Amtseinführung":"Funktionsausübung",
                  "Amtsantritt":"Funktionsausübung",
                  "Vereidigung":"Funktionsausübung",
                  "Amtsantritt":"Funktionsausübung",
                  "Beförderung":"Funktionsausübung"}

event_ending_dict= {"Entlassung":"Funktionsausübung",
                  "Suspendierung":"Funktionsausübung",
                  "Absetzung":"Funktionsausübung",
                  "Resignation":"Funktionsausübung",
                  "Rücktritt":"Funktionsausübung",
                  "Pensionierung":"Funktionsausübung",
                  "Prüfungsverfahren":"Studium",
                  "Absetzung":"Funktionsausübung",
                  "Graduation":"Studium"}

**Step 3** of the factoid consolidation includes the following operations:

*   identify exact dates resulting from identical range dates
*   add process events where only one-time events exist at present (see dictionary or events created in **step 2**)

As a result of these operations, the dataframe will become larger as additional rows are added.




In [ ]:
# open updated file with new person names from Github and create new dataframe

factoid_path_PROFs="https://github.com/ieg-dhr/DigiKAR/blob/main/Sample%20Data/Factoid_PROFS_consolidation_STEP1_merged_names-fixed.xlsx?raw=true"

df2 = pd.read_excel(factoid_path_PROFs, index_col=None, dtype=str) # axis=1, sort=False sheet_name='FactoidList'
#df2 = df2.fillna("n/a") # replace empty fields for string
df_size=len(df2)

# add additional dates where applicable

try:
  for x in range(0, df_size):
      print(df_size - x)
      e_df=df2.iloc[[x]].fillna("@") # virtual value to avoid issues with empty data frames

      # SAME START/AFTER AND END/BEFORE DATE = EXACT DATE
      # be careful to avoid chain-indexing in dataframe
      # use multi-axis indexing (df.loc['a', '1']) instead


      #f_unique['event_date'] = np.where(f_unique.event_start == f_unique.event_end, f_unique.event_start,
                      #(np.where(f_unique.event_after-date== f_unique.event_before-date, f_unique.event_after-date,
                      #<some_default_value>))

      if e_df['event_start'].equals(e_df['event_end']):
        new_date=e_df['event_start'].values[0]
        #print(new_date)
        df2.loc[x, 'event_date']=new_date
      if e_df['event_after-date'].equals(e_df['event_before-date']):
        new_date=e_df['event_after-date'].values[0]
        #print(new_date)
        df2.loc[x, 'event_date']=new_date
      if len(e_df["event_date"].values[0])>=4:
        new_date=e_df["event_date"].values[0]
        df2.loc[x, 'event_start']=new_date
        df2.loc[x, 'event_end']=new_date
      else:
        new_date=e_df['event_date'].values[0]
        df2.loc[x, 'event_date']=new_date

      # QUESTION @Florian: also create "before" and "after" dates for PROCESS EVENTS
      # if only "start" and "end" dates are available so far?

      # MATCH ONE-TIME EVENTS WITH PROCESS EVENTS

      e_value=e_df["event_type"].values[0]
      if e_value in event_starting_dict:
          #print("This event starts a process: ", e_value)
          new_event=event_starting_dict[e_value]
          if len(str(e_df['event_date'].values[0]))>=4:
            new_start=e_df['event_date'].values[0]
            new_after="n/a"
            new_end="n/a"
          elif len(str(e_df['event_start'].values[0]))>=4:
            new_after=e_df['event_start'].values[0]
            new_start="n/a"
            new_end="n/a"
          elif len(str(e_df['event_end'].values[0]))>=4:
            new_start=e_df['event_after-date'].values[0]
            new_after="n/a"
            new_end="n/a"
          elif len(str(e_df['event_before-date'].values[0]))>=4:
            new_start=e_df['event_after-date'].values[0]
            new_after="n/a"
            new_end="n/a"
          elif len(str(e_df['event_after-date'].values[0]))>=4:
            new_after=e_df['event_after-date'].values[0]
            new_start="n/a"
            new_end="n/a"
          else:
            print("No valid date!")
            new_start="n/a"
            new_end="n/a"

          s=e_df.reindex(e_df.index.repeat(1))
          s.loc[:,'event_type']=new_event # change the value
          s.loc[:, 'factoid_ID']="reconstruction" # change the value
          s.loc[:, 'event_start']=new_start # change the value
          s.loc[:, 'event_after-date']=new_after # change the value
          s.loc[:, 'event_end']=new_end # change the value

          f_to_add.append(s)
          display(s)

      elif e_value in event_ending_dict:
          #print("This event ends a process: ", e_value)
          new_event=event_ending_dict[e_value]
          if len(str(e_df['event_date'].values[0]))>=4:
            new_end=e_df['event_date'].values[0]
            new_before="n/a"
            new_start="n/a"
          elif len(str(e_df['event_start'].values[0]))>=4:
            new_before=e_df['event_start'].values[0]
            new_end="n/a"
            new_start="n/a"
          elif len(str(e_df['event_end'].values[0]))>=4:
            new_end=e_df['event_before-date'].values[0]
            new_before=e_df['event_before-date'].values[0]
            new_start="n/a"
          elif len(str(e_df['event_before-date'].values[0]))>=4:
            new_before=e_df['event_after-date'].values[0]
            new_end="n/a"
            new_start="n/a"
          elif len(str(e_df['event_after-date'].values[0]))>=4:
            new_before=e_df['event_after-date'].values[0]
            new_end="n/a"
            new_start="n/a"
          else:
            print("No valid date!")
            new_end="n/a"
            new_start="n/a"

          s=e_df.reindex(e_df.index.repeat(1))
          s.loc[:,'event_type']=new_event # change the value
          s.loc[:, 'factoid_ID']="reconstruction" # change the value
          s.loc[:, 'event_start']=new_start # change the value
          s.loc[:, 'event_after-date']=new_after # change the value
          s.loc[:, 'event_end']=new_end # change the value

          f_to_add.append(s)
          display(s)

      else:
          print("No matching event.")

except Exception as e:
  print(e)

display(f_to_add)
print(len(f_to_add))

In **step 4**, the updated data are written to an EXCEL file as a back-up and for archiving. In **steps 5 & 6**, the data will be manipulated further.

In [ ]:
from pandas.tseries.offsets import FY5253
# write amended rows to existing data frame for further processing

f_to_add.append(df2)

df3 = pd.concat(f_to_add, axis=0, ignore_index=True, sort=False)

print(len(df3))

display(df3)

workbook=directory+'FACTOIDS_consolidated/Factoid_PROFS_consolidation_STEP2_events-reconstructed_BACKUP.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
df3.to_excel(writer, sheet_name='FactCons1') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")

In the **step 5** of the consolidation, the script will try and identify similar events per person based on event type, place and institution. Those events are "aggregated" (that is: merged) while all source information etc. is preserved. In terms of dates, minimum and maximum dates given for the presumed identical events are used to create a new dataframe.

The result of this process will be that the number of rows in our table structure will be more or less drastically decreased. Where the automated factoid aggregation based on four values is too radical, more columns can be included as obligatory matches before the data are merged.

An alternative to working with the in-built aggregation function in Python is to create a hand-coded dictionary of event hierarchies and to use biographical proximities between events to decide which events are identical or succeed each other.

In [ ]:
# consolidate events per person

f_new=df3

# read person list

pers_name_f=(f_new[['pers_name']])
search_df=pers_name_f.drop_duplicates() # remove duplicates
search_list=search_df['pers_name'].tolist()

# count no. of entries in flattened person list

no_person=len(search_list)
print("There are", no_person, "unique person names in this data set.")

# iterate through unique persons to get their events

frame_list=[]
for name in search_list:
    #print("\n",name, "\n")
    res_df=(f_new.loc[f_new['pers_name'] == name])

# list existing events per person
    bio_events=res_df['event_type'].values.tolist()
    #print(set(bio_events))

# check if duplicate events with same place and institution have different dates and create range

    duplicate = res_df[res_df.duplicated(['event_type', 'pers_function', 'place_name', 'inst_name'])]
    print(duplicate)
    if len(duplicate)>1:
        print("For ", name, "there are ", len(duplicate), "similar events.")

# aggregate similar events
        try:
          df_new = duplicate.groupby(["event_type", "place_name", "inst_name"]).agg( # This line of code merges cells!!
                                        {"event_after-date":'min',
                                        "event_before-date":'max',
                                        "event_start":'min',
                                        "event_end":'min',
                                        "factoid_ID": list, # ORIGINAL IDS are combined as RECONSTRUCTION MARKER
                                        "pers_ID":list,
                                        "pers_name":list,
                                        "alternative_names":list,
                                        "pers_title":list,
                                        "pers_function":list,
                                        "inst_name":list,
                                        "rel_pers":list,
                                        "source_quotations":list,
                                        "additional_info":list,
                                        "comment":list,
                                        "info_dump":list,
                                        "source":list,
                                        "source_site":list})
          frame_list.append(df_new)
        except TypeError:
          print("One of the date fields contains invalid characters / is string!")
    else:
      continue

frame_list.append(f_new)

f_result = pd.concat(frame_list, axis=0, ignore_index=False, sort=False)


In [ ]:
# add event values from dict to data frame

try:
    f_result['event_value'] = f_result['event_type'].map(event_value_dict)
    f_result.sort_values(by =['event_after-date','event_start','event_before-date', 'event_end', 'event_value'])
except:
    print("No values.")

print("Aggregation complete!")

display(f_result)

# find events with no dates at all and reconstruct before & after dates based on hierarchies


The final step is to write the results to a single output file.

In [ ]:
# write all results to new EXCEL file

workbook=directory+'FACTOIDS_consolidated/Factoid_PROFS_consolidation_STEP3_aggregation-hierarchisation_NEW.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
f_result.to_excel(writer, sheet_name='FactCons') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")

Check the output files and repeat process if necessary.

Script by Monika Barget, Maastricht/Mainz

January 2023
